In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'
eur_usd_rate = 1.2

# Input files with new teriffs

file_smart_new_tariffs = 'SMART_190808.csv'
file_smart_new_tariffs_data = 'SMART_DATA_190808.csv'

In [2]:
def clean_smart_file(folder, file, tariff_type):
    # tariff_type "data" / "all"
    df_smart = pd.read_csv(join(folder, file),sep=',')
    df_smart.Data_Increment_KB.replace({'1 kB':1, '10 kB':10, '1kB':1, '1kb':1, '10kb':10, ' 10kb ':10}, inplace=True)
    df_smart.LTE_S5.replace({'3G':'No', '4G':'Yes'}, inplace=True)
    df_smart.Data_Increment_KB.fillna(10, inplace=True)
    df_smart.Voice_Increment_SEC.fillna(60, inplace=True)
    
    # #####################################################
    #Выбрать только первый TADIG из ячейки, если их там несколько через запятую
    # #####################################################

    df_smart.TADIG = df_smart.TADIG.str.extract('([A-Z0-9]+)', expand=True)
    
    # #####################################################
    
    if tariff_type == 'data':
        df_smart.MOC.fillna(50, inplace=True)
        df_smart.MTC.fillna(50, inplace=True)
        df_smart.SMS.fillna(50, inplace=True)
    elif tariff_type == 'all':
        df_smart.MTC.fillna(0, inplace=True)
        
        df_smart.loc[df_smart.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MOC'] =\
        df_smart.MOC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate

        df_smart.loc[df_smart.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MTC'] =\
        df_smart.MTC.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate

        df_smart.loc[df_smart.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'SMS'] =\
        df_smart.SMS.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate

    # #####################################################
    # Очистить данные от EUR с конвертацией в USD
    # #####################################################

    df_smart.loc[df_smart.MB.str.extract('(EUR\.?) (.*)', expand=True)[1].notnull(),'MB'] =\
    df_smart.MB.str.extract('(EUR\.?) (.*)', expand=True)[1].astype('float64') * eur_usd_rate

    df_smart[['MOC','MTC','SMS','MB']] = df_smart[['MOC','MTC','SMS','MB']].replace(to_replace={r'-':0}, regex=True)

    df_smart[['MOC','MTC','SMS','MB']] = df_smart[['MOC','MTC','SMS','MB']].astype('float64')
    
    return df_smart

In [3]:
df_smart = clean_smart_file(load_folder, file_smart_new_tariffs, 'all')
display(df_smart.head(3))

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5
0,CAMEROON,MTN Cameroon,CMRMT,0.50,0.0,0.030,0.0100,60.0,1,No
1,GHANA,VODAFONE,GHAGT,0.14,0.0,0.005,0.0075,60.0,1,No
2,IVORY COAST,MTN Ivory Coast,CIVTL,0.50,0.0,0.030,0.0100,60.0,1,No


In [4]:
df_smart_data = clean_smart_file(load_folder, file_smart_new_tariffs_data, "data")
display(df_smart_data.head(3))

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5
0,AFGHANISTAN,AFGHAN WIRELESS,AFGAW,50.0,50.0,50.0,0.1000,60.0,1.0,No
1,AFGHANISTAN,ETISALAT,AFGEA,50.0,50.0,50.0,0.0150,60.0,1.0,No
2,ALBANIA,VODAFONE ALBANIA,ALBVF,50.0,50.0,50.0,0.0075,60.0,1.0,No


In [5]:
df_smart_all = df_smart.append(df_smart_data[~df_smart_data.TADIG.isin(df_smart.TADIG.tolist())], ignore_index=True)
df_smart_all.to_csv(join(load_folder, 'SMART_BASE.csv'), index=False)
df_smart_all.head(3)

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5
0,CAMEROON,MTN Cameroon,CMRMT,0.50,0.0,0.030,0.0100,60.0,1.0,No
1,GHANA,VODAFONE,GHAGT,0.14,0.0,0.005,0.0075,60.0,1.0,No
2,IVORY COAST,MTN Ivory Coast,CIVTL,0.50,0.0,0.030,0.0100,60.0,1.0,No


In [6]:
df_smart_all_15 = df_smart_all.copy()
df_smart_all_15[['MOC','MTC','SMS','MB']] = df_smart_all_15[['MOC','MTC','SMS','MB']] * 1.15
df_smart_all_15.to_csv(join(load_folder, 'SMART_15.csv'), index=False)
df_smart_all_15.head(3)

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5
0,CAMEROON,MTN Cameroon,CMRMT,0.575,0.0,0.03450,0.011500,60.0,1.0,No
1,GHANA,VODAFONE,GHAGT,0.161,0.0,0.00575,0.008625,60.0,1.0,No
2,IVORY COAST,MTN Ivory Coast,CIVTL,0.575,0.0,0.03450,0.011500,60.0,1.0,No


In [7]:
df_smart_all_100 = df_smart_all_15.copy()
df_smart_all_100[['MOC','MTC','SMS','MB']] = df_smart_all_100[['MOC','MTC','SMS','MB']] * 2
df_smart_all_100.to_csv(join(load_folder, 'SMART_100.csv'), index=False)
df_smart_all_100.head(3)

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,LTE_S5
0,CAMEROON,MTN Cameroon,CMRMT,1.150,0.0,0.0690,0.02300,60.0,1.0,No
1,GHANA,VODAFONE,GHAGT,0.322,0.0,0.0115,0.01725,60.0,1.0,No
2,IVORY COAST,MTN Ivory Coast,CIVTL,1.150,0.0,0.0690,0.02300,60.0,1.0,No
